### Chat bot with conversation history

https://reference.langchain.com/python/langchain-core/messages/human/HumanMessage
https://docs.langchain.com/oss/python/langchain/messages


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi my name is dennis, i'm a chief AI engineer")])

AIMessage(content="Hello Dennis! It's a pleasure to meet you. As a chief AI engineer, you're likely at the forefront of some fascinating work. I'd love to hear more about your current projects or any challenges you're tackling in the field. Whether it's about machine learning models, AI ethics, or the latest advancements in neural networks, I'm here to engage in meaningful discussions and explore ideas together. What's on your mind today?", additional_kwargs={'reasoning_content': "Okay, the user introduced himself as Dennis, a chief AI engineer. First, I need to respond warmly and show respect for his professional background. I should ask about his specific work content or challenges he's facing to better understand his needs. At the same time, I need to keep the conversation open-ended to encourage him to share more information. I should also consider his possible needs, such as technical support, project collaboration, or career development advice. I need to maintain a friendly and p

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(content="Hi my name is dennis, i'm a chief AI engineer"),
        AIMessage(
            content="Hello Dennis! It's a pleasure to meet you. As a Chief AI Engineer, you're likely at the forefront of some fascinating projects. I'd love to hear more about your current work—whether it's in machine learning, natural language processing, computer vision, or any emerging AI technologies. \n\nAre you working on any specific challenges or innovations that you'd like to discuss? Whether it's technical hurdles, ethical considerations, or strategic implementations, I'm here to engage in a meaningful dialogue. Collaboration and knowledge-sharing are key in our field, so feel free to dive into any topic that interests you!",
            additional_kwargs={
                "reasoning_content": "Okay, the user introduced himself as Dennis, a chief AI engineer. First, I need to acknowledge his introduction and express interest in his work. Since he's in a technical field, maybe he's looking for technical discussions or advice. I should ask about his current projects or challenges he's facing. Also, considering his role, he might be interested in collaboration opportunities or sharing knowledge. I should keep the conversation open-ended to let him decide the direction. Need to make sure the response is friendly and professional, matching his level of expertise. Maybe mention specific areas like machine learning, deep learning, or AI ethics if he's interested. Avoid being too generic. Let him know I'm here to help with any specific questions he might have.\n"
            },
            response_metadata={
                "token_usage": {
                    "completion_tokens": 274,
                    "prompt_tokens": 21,
                    "total_tokens": 295,
                    "completion_time": 0.564555356,
                    "completion_tokens_details": {"reasoning_tokens": 148},
                    "prompt_time": 0.0007223,
                    "prompt_tokens_details": None,
                    "queue_time": 0.04696852,
                    "total_time": 0.565277656,
                },
                "model_name": "qwen/qwen3-32b",
                "system_fingerprint": "fp_5cf921caa2",
                "service_tier": "on_demand",
                "finish_reason": "stop",
                "logprobs": None,
                "model_provider": "groq",
            },
            id="lc_run--019c95a1-a61e-70d3-855e-2d7309db73a3-0",
            usage_metadata={
                "input_tokens": 21,
                "output_tokens": 274,
                "total_tokens": 295,
                "output_token_details": {"reasoning": 148},
            },
        ),
        HumanMessage(content="What is my name and what do i do?"),
    ]
)

AIMessage(content='Your name is **Dennis**, and you are a **Chief AI Engineer**. You’re likely involved in designing, developing, and overseeing AI systems, whether that’s building machine learning models, leading teams, or innovating in areas like NLP, computer vision, or AI ethics. \n\nIf you’d like to dive deeper into any specific projects, challenges, or ideas related to your work, feel free to share—I’m here to help brainstorm, troubleshoot, or explore concepts with you! 😊', additional_kwargs={'reasoning_content': 'Okay, the user is asking, "What is my name and what do I do?" Let me check the conversation history.\n\nIn the previous message, the user introduced themselves as Dennis, a Chief AI Engineer. The assistant acknowledged that and asked about their work. Now, Dennis is asking again for confirmation of their name and role. \n\nHmm, maybe Dennis wants to ensure that the assistant correctly remembers their information. It\'s possible they\'re testing if the AI can retain cont

### session management & message history

https://medium.com/@jkSurampudi5/message-history-in-langchain-8b6c10e89e60


In [7]:
# Writing function to retrive chat history for a specific session_id

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


# function to retrive chat history for a specific session_id
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [23]:
# specify the session configuration
config1 = {"configurable": {"session_id": "chat1"}}

In [ ]:
# invoke the function
response1 = with_message_history.invoke(  # the meaning of this line is to call the model with message history
    [HumanMessage(content="My name is dennis. I am a AI developer")],
    config=config1,
)
response1.content

"Hi Dennis! Nice to meet you. As an AI developer, you likely work on machine learning models, neural networks, or AI-driven solutions. How can I assist you today? Whether it's about model training, deployment, data preprocessing, AI ethics, tools like TensorFlow/PyTorch, or anything else in the AI/ML space—feel free to share what you're working on! 😊"

In [27]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1,
)
response2.content

'Hi Dennis! Your name is **Dennis**, as you mentioned earlier. 😊 Let me know how I can assist you with your AI development work!'

In [28]:
print(store["chat1"].messages)

[HumanMessage(content='Hi my name is Krishna. I am a Data Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Krishna! Nice to meet you. As a Data Engineer, you probably work with data pipelines, ETL processes, and infrastructure. How can I assist you today? Whether it's about data engineering challenges, tools, best practices, or anything else related to your work, feel free to ask!", additional_kwargs={'reasoning_content': "Okay, the user introduced themselves as Krishna, a Data Engineer. I should acknowledge their introduction and offer assistance. Let me make sure to respond warmly and ask how I can help them today. I'll keep it friendly and open-ended.\n"}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 19, 'total_tokens': 131, 'completion_time': 0.244956983, 'completion_tokens_details': {'reasoning_tokens': 47}, 'prompt_time': 0.000655941, 'prompt_tokens_details': None, 'queue_time': 0.046034125, 'total_time': 0.245612924},

In [29]:
# change the session config and ask for your name again
config2 = {"configurable": {"session_id": "chat2"}}

response1 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config2,
)
response1.content

"I don't have access to your personal information, including your name. If you'd like me to address you by name in our conversation, feel free to share it with me!"

### Working with chat prompt template

https://reference.langchain.com/python/langchain-core/prompts/chat/MessagesPlaceholder


In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),  # history is a variable
    ]
)

chain = prompt | model

In [31]:
chain.invoke({"history": [HumanMessage(content="Hi my name is dennis")]})

AIMessage(content="Hi Dennis! Nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if there's something specific you'd like help with. 😊", additional_kwargs={'reasoning_content': "Okay, the user introduced themselves as Dennis. I should acknowledge their name and offer assistance. Let me make sure to use their name in the response to personalize the interaction. I'll keep it friendly and open-ended to encourage them to ask questions or share what's on their mind. Let me check for any typos and ensure the tone is welcoming.\n"}, response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 25, 'total_tokens': 139, 'completion_time': 0.221469843, 'completion_tokens_details': {'reasoning_tokens': 70}, 'prompt_time': 0.000832029, 'prompt_tokens_details': None, 'queue_time': 0.04697828, 'total_time': 0.222301872}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'sto

In [41]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [43]:
config3 = {"configurable": {"session_id": "chat3"}}

response1 = with_message_history.invoke(
    {"history": [HumanMessage(content="Hi my name is dennis")]},
    config=config3,
)
response1.content

"Hello again, Dennis! 😊 I'm glad to see you here. As an AI developer, I'd love to hear about your projects or discuss any challenges you're working on. Whether it's machine learning, NLP, computer vision, or something else—feel free to share! How can I assist you today? 🚀"

In [21]:
# prompt for translation

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}",
        ),
        MessagesPlaceholder("history"),
    ]
)

chain = prompt | model

In [ ]:
response = chain.invoke(
    {"history": [HumanMessage(content="Hi my name is dennis")], "language": "hindi"}
)
response.content

'हैलो डेनिस! मैं आपकी कैसे मदद कर सकता हूँ?'

In [50]:
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="history"
)

In [ ]:
config4 = {"configurable": {"session_id": "chat4"}}

response = with_message_history.invoke(
    {"history": [HumanMessage(content="Hi my name is dennis")], "language": "hindi"},
    config=config4,
)
response.content

'नमस्ते डेनिस! 😊 कैसे मदद कर सकता हूँ आपको? कोई प्रश्न है या कुछ जानना चाहते हैं?'

In [52]:
response = with_message_history.invoke(
    {"history": [HumanMessage(content="What is my name?")], "language": "hindi"},
    config=config4,
)
response.content

'आपका नाम डेनिस है। 😊 क्या आपको कुछ और बताने की आवश्यकता है?'

### Managing Chat Histories

https://reference.langchain.com/python/langchain-core/messages/utils/trim_messages


In [22]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    BaseMessage,
    SystemMessage,
    trim_messages,
)

messages = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just '
        "didn't have the same ring to it!"
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last "
        "cup of coffee in the office!"
    ),
    HumanMessage("what do you call a speechless parrot"),
    AIMessage("A speechless bird"),
]

trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    start_on="human",
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    allow_partial=False,
)

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and who is harrison chasing anyways', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what do you call a speechless parrot', additional_kwargs={}, response_metadata={}),
 AIMessage(content='A speechless bird', additional_kwargs={}, response_metadata={})]

In [23]:
trimmer = trim_messages(
    max_tokens=45,  # reducing the number of tokens
    strategy="last",
    token_counter=model,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    start_on="human",
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    allow_partial=False,
)

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what do you call a speechless parrot', additional_kwargs={}, response_metadata={}),
 AIMessage(content='A speechless bird', additional_kwargs={}, response_metadata={})]

In [35]:
# invoking through chain

from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("history") | trimmer)
    | prompt
    | model
)
# prompt : refers to prompt template above
# history : refers to the variable passed in prompt template

response = chain.invoke(
    {
        "history": messages + [HumanMessage(content="what question did i ask")],
        "language": "English",
    }
)
response.content

'You asked, "What do you call a speechless parrot?" and now you\'re asking about your own questions? That’s a real brain teaser! 😄'

In [32]:
# trying to access a trimmed message : it will give me a random answer and not the answer from messages.
response = chain.invoke(
    {
        "history": messages + [HumanMessage(content="What is the chasing question")],
        "language": "English",
    }
)
response.content

'Ah, the classic "Why did the chicken cross the road?" question—because it was tired of being chased by the egg! 🐔🥚'

In [36]:
# wrap in message history
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history, 
    input_messages_key="history"
)
config = {"configurable": {"session_id": "chat5"}}

In [37]:
response = with_message_history.invoke(
    {"history": [HumanMessage(content="What is the question asked?")], "language": "English"},
    config=config,
)
response.content

'You’re asking for clarification about a question that hasn’t been provided in our conversation. To assist you effectively, please share the **specific question** you’d like me to answer. Without a clear question, I can’t provide a meaningful response. Let me know how I can help! 😊'